In [1]:
import nltk
import pandas as pd
import numpy as np
import re
import codecs
import seaborn as sns
import matplotlib.pyplot as plt
from bnltk.tokenize import Tokenizers
import string
import pickle


#  tensorflow
import tensorflow  as tf
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# sklearn
from sklearn.model_selection import train_test_split

In [2]:
print(tf.__version__)

2.15.0


In [3]:
# load dataset
df = pd.read_excel('./dataset/data_1500_Reviews.xlsx', index_col=0)

In [4]:
#load stopwords
stopword = codecs.open("./dataset/stopwords-bn.txt", 'r', encoding='utf-8').read().split()

In [5]:
stopword[:10]

['অতএব',
 'অথচ',
 'অথবা',
 'অনুযায়ী',
 'অনেক',
 'অনেকে',
 'অনেকেই',
 'অন্তত',
 'অন্য',
 'অবধি']

In [6]:
df[:10]

,Reviews,Sentiment
0,পিজাটা চমৎকার ছিল । আমি এবং আমার বন্ধুদের এটি ...,positive
1,এরাবিয়ান মাস্টারের অভ্যন্তরীণ সজ্জাটা অনন্য। ...,positive
2,কোরিয়ান এবং জামাইকান BBQ উপভোগ করার জন্য একটি...,positive
3,BBQ এর মেনুগুলো সেরা।,positive
4,শ্রেষ্ঠ স্বাদ ..... খাবারের মানও ভাল ....,positive
5,সেরা পরিবেশ সঙ্গে শ্রেষ্ঠ স্বাদ। একেবারে প্রিম...,positive
6,সত্যিই খেতে দারুণ টেস্ট !!! আমি শুধু BBQ ভালব...,positive
7,মহান খাদ্য বাহ ভাল বাহ সঙ্গে মহান সেবা ..........,positive
8,অসাধারণ এবং সুস্বাদু খাদ্য ... ভাল সার্ভিস এবং...,positive
9,গরম এবং মসলাযুক্ত চিকেন ফ্রাইটা পছন্দনীয়। চমৎক...,positive


In [7]:
df['Sentiment'].value_counts()

Sentiment
negative    794
positive    637
Name: count, dtype: int64

In [8]:
df.isnull().sum()

Reviews      0
Sentiment    0
dtype: int64

#### Preprocessing 

In [9]:
punctuation_marks = set(string.punctuation)

corpus = []

for i in range(0, len(df)):

    # remove english word
    reviews = re.sub(r'\b[a-zA-Z]+\b', '',  df['Reviews'].iloc[i])
    # tokenize
    words = Tokenizers.bn_word_tokenizer(reviews)
    # remove punctuation marks
    reviews = [word for word in words if word not in punctuation_marks]
    # remove bangla stopwords
    reviews = [word for word in reviews if word not in stopword]
    reviews = " ".join(reviews).strip()
    corpus.append(reviews)


In [10]:
corpus[:10]

['পিজাটা চমৎকার । বন্ধুদের পছন্দ হয়েছে ।',
 'এরাবিয়ান মাস্টারের অভ্যন্তরীণ সজ্জাটা অনন্য । এটির খাদ্য সুস্বাদু একটু মসলাযুক্ত কর্মীরা বন্ধুত্বপূর্ণ ।',
 'কোরিয়ান জামাইকান উপভোগ চমৎকার জায়গা ।',
 'মেনুগুলো সেরা ।',
 'শ্রেষ্ঠ স্বাদ খাবারের মানও ভাল',
 'সেরা পরিবেশ শ্রেষ্ঠ স্বাদ । একেবারে প্রিমিয়াম মুরগি ।',
 'সত্যিই খেতে দারুণ টেস্ট শুধু ভালবাসি । খাবারটা অসাধারণ । জায়গায় ভালোবাসি । ভাল ।',
 'মহান খাদ্য বাহ ভাল বাহ মহান সেবা',
 'অসাধারণ সুস্বাদু খাদ্য ভাল সার্ভিস নিশ্চিত জায়গাটা সেরা কোরিয়ান ভালোবাসি ।',
 'গরম মসলাযুক্ত চিকেন ফ্রাইটা পছন্দনীয় । চমৎকার পরিবেশ আতিথেয়তা । এটিকে ঢাকায় শীর্ষস্থানীয় রেস্তোরাঁর রেটিং দিব ।']

In [11]:
df['Sentiment']

0       positive
1       positive
2       positive
3       positive
4       positive
          ...   
1426    negative
1427    negative
1428    negative
1429    negative
1430    negative
Name: Sentiment, Length: 1431, dtype: object

In [12]:


len(df[df['Sentiment'] == 'positive']), len(df[df['Sentiment'] == 'negative'])

(637, 794)

In [13]:
# df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

df['Sentiment'] = df['Sentiment'].map({"positive": 1, "negative": 0})

In [14]:
df['Sentiment']

0       1
1       1
2       1
3       1
4       1
       ..
1426    0
1427    0
1428    0
1429    0
1430    0
Name: Sentiment, Length: 1431, dtype: int64

In [15]:
df['Sentiment'].value_counts()

Sentiment
0    794
1    637
Name: count, dtype: int64

In [16]:
# count all words
tokenizer = Tokenizer()
# fit_on_texts() => creates the vocabulary and counts how many times each word occurs. It calculates the frequency of each word and then makes an index list for all the words.
tokenizer.fit_on_texts(corpus)
# Get total number of unique words
total_words = len(tokenizer.word_index)+1
total_words

4051

In [17]:
tokenizer.word_index

{'।': 1,
 'খাদ্য': 2,
 'ভাল': 3,
 'না': 4,
 'খারাপ': 5,
 'খাবার': 6,
 'জায়গা': 7,
 'সেবা': 8,
 'স্বাদ': 9,
 '0': 10,
 'চমৎকার': 11,
 'মান': 12,
 'পরিবেশ': 13,
 'শুধু': 14,
 'ভালো': 15,
 'খাবারের': 16,
 '1': 17,
 'সত্যিই': 18,
 'সুস্বাদু': 19,
 'সার্ভিস': 20,
 'মূল্য': 21,
 'সময়': 22,
 'মানের': 23,
 'পছন্দ': 24,
 'চিকেন': 25,
 'মত': 26,
 'মুরগির': 27,
 '√': 28,
 'খুবই': 29,
 'আচরণ': 30,
 'পিজা': 31,
 'এক': 32,
 'নয়': 33,
 'সাথে': 34,
 'বার্গার': 35,
 'সেরা': 36,
 'অভিজ্ঞতা': 37,
 '2': 38,
 'কখনো': 39,
 'সবচেয়ে': 40,
 'সুন্দর': 41,
 'দাম': 42,
 'টাকা': 43,
 'আদেশ': 44,
 'ô': 45,
 'অসাধারণ': 46,
 '5': 47,
 'মহান': 48,
 'অর্ডার': 49,
 'ঠিক': 50,
 'এমনকি': 51,
 'পরিমাণ': 52,
 'চেষ্টা': 53,
 'হতাশ': 54,
 'ø': 55,
 'টেস্ট': 56,
 '3': 57,
 'প্লেটার': 58,
 'রেস্টুরেন্টে': 59,
 'গিয়েছিলাম': 60,
 'ছোট': 61,
 'বেশি': 62,
 'আরো': 63,
 'কফি': 64,
 'মেনু': 65,
 'ওয়েটার': 66,
 'কম': 67,
 'পরিবেশন': 68,
 'এছাড়াও': 69,
 'আইটেম': 70,
 'ω': 71,
 '4': 72,
 'ভয়ঙ্কর': 73,
 'সম্পর্কে': 74,
 'যথেষ্ট': 

In [18]:
tokenizer.word_index['খাদ্য']

2

In [19]:
tokenizer = Tokenizer(num_words=total_words) 
tokenizer.fit_on_texts(corpus)
# creates a list using the word_index of each word.
X = tokenizer.texts_to_sequences(corpus)


In [20]:
X[:20]

[[276, 11, 1, 202, 24, 130, 1],
 [624, 1256, 529, 1855, 405, 1, 938, 2, 19, 156, 121, 218, 113, 1],
 [108, 277, 163, 11, 7, 1],
 [1856, 36, 1],
 [86, 9, 16, 372, 3],
 [36, 13, 86, 9, 1, 164, 939, 240, 1],
 [18, 182, 755, 56, 14, 303, 1, 122, 46, 1, 530, 131, 1, 3, 1],
 [48, 2, 625, 3, 625, 48, 8],
 [46, 19, 2, 3, 20, 165, 626, 36, 1857, 131, 1],
 [183,
  121,
  25,
  1858,
  241,
  1,
  11,
  13,
  193,
  1,
  1257,
  256,
  940,
  756,
  373,
  1258,
  1],
 [1859, 1, 1],
 [332, 46, 1, 757, 1],
 [18,
  3,
  7,
  1,
  1259,
  1860,
  3,
  1,
  36,
  193,
  1,
  2,
  3,
  1,
  166,
  2,
  18,
  627,
  219,
  21,
  1,
  220,
  758,
  194,
  167,
  3,
  1,
  374,
  531,
  41,
  532,
  24,
  130,
  1],
 [757, 6, 143, 20],
 [628,
  18,
  3,
  7,
  132,
  108,
  25,
  1861,
  1862,
  3,
  1260,
  276,
  459,
  1,
  941,
  15,
  1261],
 [3, 6, 11, 8, 1],
 [1863, 32, 942, 1864, 629, 209, 46, 1865, 16, 1262],
 [131],
 [19, 1, 122, 18, 3, 1, 278, 1866, 46, 1],
 [19]]

In [21]:
# save bangla tokenizer
with open("bangla_tokenizer.pkl", 'wb') as file:
    pickle.dump(tokenizer, file)

In [22]:
# Pad sequences
max_length = max([len(i) for i in X])
max_length

149

In [23]:
X = pad_sequences(X, maxlen=max_length, padding="pre")
X

array([[   0,    0,    0, ...,   24,  130,    1],
       [   0,    0,    0, ...,  218,  113,    1],
       [   0,    0,    0, ...,   11,    7,    1],
       ...,
       [   0,    0,    0, ..., 1854,  426,    1],
       [   0,    0,    0, ...,  186, 1095,    4],
       [   0,    0,    0, ...,  174, 4050,    1]])

In [24]:
y = df['Sentiment']

In [25]:
# split the dataset into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
len(X_train), len(y_train), len(X_test), len(y_test)

(1144, 1144, 287, 287)

In [27]:
print(total_words)
print(max_length)

4051
149


In [28]:
 
# Build model
model = Sequential()
# Embedding layer for use word_index to convert word embedding
model.add(Embedding(input_dim=total_words, output_dim=128, input_length=max_length))
# first hidden layer
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2)) # dropout 20% neuron for reduce overfitting 
# second hidden layer
model.add(LSTM(64))
model.add(Dropout(0.2))
# output layer
model.add(Dense(1, activation="sigmoid")) 




In [29]:
model.build(input_shape=(None, max_length))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 149, 128)          518528    
                                                                 
 lstm (LSTM)                 (None, 149, 128)          131584    
                                                                 
 dropout (Dropout)           (None, 149, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 699585 (2.67 MB)
Trainable params: 699585 

In [30]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [31]:
# setup earlystoping
early_stoping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [32]:
# train the model
model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stoping]
)

Epoch 1/100




18/18 [==============================] - 8s 218ms/step - loss: 0.6376 - accuracy: 0.6512 - val_loss: 0.6121 - val_accuracy: 0.6655
Epoch 2/100
18/18 [==============================] - 3s 163ms/step - loss: 0.4546 - accuracy: 0.7867 - val_loss: 0.4383 - val_accuracy: 0.7979
Epoch 3/100
18/18 [==============================] - 3s 162ms/step - loss: 0.2180 - accuracy: 0.9283 - val_loss: 0.3502 - val_accuracy: 0.8815
Epoch 4/100
18/18 [==============================] - 3s 161ms/step - loss: 0.1258 - accuracy: 0.9580 - val_loss: 0.4067 - val_accuracy: 0.8780
Epoch 5/100
18/18 [==============================] - 3s 158ms/step - loss: 0.0605 - accuracy: 0.9790 - val_loss: 0.4494 - val_accuracy: 0.8746
Epoch 6/100
18/18 [==============================] - 3s 162ms/step - loss: 0.0318 - accuracy: 0.9913 - val_loss: 0.5198 - val_accuracy: 0.8780
Epoch 7/100
18/18 [==============================] - 3s 159ms/step - loss: 0.0202 - accuracy: 0.9956 - val_loss: 0.5507 - val_accuracy: 0.8641
Epoch 8/1

In [37]:
# Evaluation
test_loss, test_acc = model.evaluate(X_test, y_test) 

9/9 [==============================] - 1s 36ms/step - loss: 0.3502 - accuracy: 0.8815


In [38]:
# save the model
model.save("bangla_lstm.h5")

In [39]:
# Load model and tokenizer
from tensorflow.keras.models import load_model

model = load_model("bangla_lstm.h5")
with open("bangla_tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

In [40]:
# Text preprocessing function
def preprocess_text(text):
    reviews = re.sub(r'\b[a-zA-Z]+\b', '', text)
    # tokenize
    words = Tokenizers.bn_word_tokenizer(reviews)
    # remove punctuation marks
    reviews = [word for word in words if word not in punctuation_marks]
    # remove bangla stopwords
    reviews = [word for word in reviews if word not in stopword]
    cleaned_text = " ".join(reviews).strip()
    encoded_review = tokenizer.texts_to_sequences([cleaned_text])
    padded_review = pad_sequences(encoded_review, maxlen=max_length)
    return padded_review
    

In [41]:
# Predict function
def predict_sentiment(review):
    processed_input = preprocess_text(review)
    prediction = model.predict(processed_input)
    sentiment = 'Positive' if prediction[0][0] > 0.6 else 'Negative'
    return sentiment, prediction[0][0]

In [43]:
# Example
example_review = "সার্ভিস খুব দ্রুত এবং দক্ষ।"
sentiment, score = predict_sentiment(example_review)

print("Review Text:", example_review)
print("Sentiment:", sentiment)
print("Score:", score)

1/1 [==============================] - 0s 31ms/step
Review Text: সার্ভিস খুব দ্রুত এবং দক্ষ।
Sentiment: Negative
Score: 0.5229931
